In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import py3Dmol

from proteome import protein
from proteome.models.design.genie.modeling import GenieForStructureDesign
from proteome.models.design.proteinmpnn.modeling import ProteinMPNNForSequenceDesign
from proteome.models.folding.omegafold.modeling import OmegaFoldForFolding

In [3]:
structure_designer = GenieForStructureDesign()
sequence_designer = ProteinMPNNForSequenceDesign("ca_only_model-20", random_seed=37)
folder = OmegaFoldForFolding()

In [4]:
structure = structure_designer.design_structure()

sampling loop time step: 100%|████████████████████████████| 1000/1000 [01:05<00:00, 15.24it/s]


In [5]:
sequence, score = sequence_designer.design_sequence(structure)
print(f"Sequence: {sequence} with global_score {score}")

Sequence: SKLLEAIIKYLEGKKNEDYFIKILKKGKKNKKIKKKLKIEIPSDLGELAEKIKKLSKKIKKKYPNIKFKLNKKGESFYLELPTSKKKCKCKPLGKELKSNSIDKIECPNKFKLPSKFYVKGKEKKEIK with global_score 1.509469747543335


In [6]:
predicted_protein, confidence = folder.fold(sequence)
result_pdb = protein.to_pdb(predicted_protein)

In [7]:
PLDDT_BANDS = [
  (0, 50, '#FF7D45'),
  (50, 70, '#FFDB13'),
  (70, 90, '#65CBF3'),
  (90, 100, '#0053D6')
]
view = py3Dmol.view(width=800, height=600)
view.addModelsAsFrames(result_pdb)

color_map = {i: bands[2] for i, bands in enumerate(PLDDT_BANDS)}
style = {'cartoon': {'colorscheme': {'prop': 'b', 'map': color_map}}}

style['stick'] = {}

view.setStyle({'model': -1}, style)
view.zoomTo()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol